In [1]:
import pandas as pd

In [12]:
import pandas as pd
import re
import emoji
import stopwordsiso as stopwordsiso
from langdetect import detect
import numpy as np

# Define supported languages and stopwords
LANGS = ["en", "de"]

def get_stopwords(langs=None, extra=None):
    """
    Build a multilingual stopword set with optional extra filler terms.
    
    Returns:
        set: A unified set of stopwords.
    """
    if langs is None:
        langs = LANGS
    if extra is None:
        extra = ["im", "like", "thing", "ok", "got", "youre", "omg", "pls", "tbh", "smh", "aint"]

    stopword_set = set()
    for lang in langs:
        words = stopwordsiso.stopwords(lang)
        if words:
            stopword_set.update(words)

    stopword_set.update(extra)
    return list(stopword_set)


def clean_text(text, stopwords=None):
    """
    Lowercase, remove URLs/mentions, normalize repeated characters,
    keep emojis, clean punctuation, and optionally remove stopwords.
    
    Args:
        text (str): Raw input text.
        stopwords (set, optional): If provided, stopwords will be removed.

    Returns:
        str: Cleaned text.
    """
    # Lowercase
    text = text.lower()

    # Remove URLs and mentions
    text = re.sub(r"http\S+|www\S+|@\w+", " ", text)

    # Keep emojis by spacing them
    text = emoji.replace_emoji(text, replace=lambda e, _: f" {e} ")

    # Remove all characters except letters, numbers, whitespace, and emojis
    text = re.sub(r"[^a-zA-ZäöüÄÖÜß0-9\s\U0001F600-\U0001F64F]+", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize and normalize repeated characters
    tokens = text.split()
    tokens = [re.sub(r"(.)\1{2,}", r"\1", t) for t in tokens]

    # Remove stopwords and short tokens
    if stopwords:
        tokens = [t for t in tokens if t not in stopwords and len(t) > 2]
    else:
        tokens = [t for t in tokens if len(t) > 2]

    return " ".join(tokens)


/Users/hannahschlaucher/Library/Caches/pypoetry/virtualenvs/skincare-analysis-pFfgqJ8c-py3.10/lib/python3.10/site-packages/stopwordsiso/_core.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
df = pd.read_csv("../data/comments_recent.csv")


df = df[df["textLanguage"] == "en"].copy()
df['comment'] = df['comment'].astype(str)

# Remove rows containing any unwanted keywords (case-insensitive)
pattern = r"sagajewels|jewelry|collaboration|collab"
df = df[~df['comment'].str.contains(pattern, case=False, na=False)]

# Save original comment
df["original_text"] = df['comment']

# Clean comments
stopwords = get_stopwords(langs=["en"])
df["cleaned_text"] = df['comment'].apply(lambda x: clean_text(x, stopwords=stopwords))

# Filter and deduplicate
df_filtered = df[df["cleaned_text"].str.split().str.len() > 1]
df_unique = df_filtered.drop_duplicates(subset=["cleaned_text"])

# Extract cleaned texts
comments = df_unique["cleaned_text"].tolist()

In [20]:
#len(df)
len(df_filtered)
#len(df_unique)

953

In [15]:
len(comments)

947

In [9]:
df_comments['createTimeISO'] = pd.to_datetime(df_comments['createTimeISO'], utc=True)

# sort newest→oldest (set ascending=False for newest first)
df_comments = df_comments.sort_values('createTimeISO', ascending=False).reset_index(drop=True)
df_comments

,post_id,text,textLanguage,createTimeISO,isAd,author_nickName,author_signature,author_fans,video_duration,diggCount,...,comment_createTimeISO_comment,comment,diggCount_comment,replyCommentTotal_comment,uniqueId_comment,detected_language,transcribed_text,video_description,date,comment_lang
0,7508707589408967958,[Anzeige] Baumwoll-Pflegetücher und Mandelöl v...,de,2025-05-26 11:00:25+00:00,False,🇺🇦Lilia in Berlin🇩🇪,🇺🇦from UA| live in Berlin \nПро життя та матер...,3559,30,37,...,2025-05-27T20:05:36.000Z,☺️☺️☺️☺️Herzlichen Glückwunsch 🍾,1,1.0,anna8lebedeva,NaN,NaN,NaN,2025-05-26,de
1,7508707415362161942,##beauty #antiaging #glow #skincareroutine #sk...,un,2025-05-26 10:59:46+00:00,False,TamGru,💗 𝓢𝓀𝓲𝓷𝓬𝓪𝓻𝓮 💗\n\n\nhttps://wonderl.ink/@tamara-...,145,27,2,...,2025-05-26T17:45:44.000Z,So schön ❤️,0,0.0,saskia.beauty.glowyskin,English,"If you laugh, you can laugh, and you will find...",No description available.,2025-05-26,de
2,7508676349607824662,"Lasst euch nicht alles andrehen, ihr seid alle...",de,2025-05-26 08:59:14+00:00,False,Leyla Hajrovic,"Content Creator Fashion, Beauty & Lifestyle\nF...",36700,55,367,...,2025-05-26T15:59:42.000Z,Das hesch du so toll xseid! Jede tag isch‘s eh...,0,1.0,positiv055,German,Viele Leute auf Social Media versuchen euch di...,Die Frau in diesem Video ist im Badezimmer und...,2025-05-26,de
3,7508676349607824662,"Lasst euch nicht alles andrehen, ihr seid alle...",de,2025-05-26 08:59:14+00:00,False,Leyla Hajrovic,"Content Creator Fashion, Beauty & Lifestyle\nF...",36700,55,367,...,2025-05-26T09:04:18.000Z,"LSF50 darüber diskutieren wir aber nicht, der ...",9,0.0,leylahajrovic,German,Viele Leute auf Social Media versuchen euch di...,Die Frau in diesem Video ist im Badezimmer und...,2025-05-26,de
4,7507593092631973142,Jetzt aber mal Klartext. 💚 #NIVEA #Skincare...,de,2025-05-26 06:30:00+00:00,False,NIVEA DE,Iconic Skincare 🤝 wissenschaftlich belegt 👩‍🔬🔬,32000,81,83,...,2025-05-26T10:04:53.000Z,Wie cool! Danke für die Aufklärung 🔝,3,1.0,larissa_sch3,German,"Kommt es, dass Nivea bei Peta und Co immer noc...",Find out why Nivea products are not PETA certi...,2025-05-26,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30456,7472277773537529110,Deinen Hauttypen solltest du unbedingt kennen ...,de,2025-02-17 06:54:17+00:00,False,MIA 🦋 I Skincare & Glow ups,🫧 Deine Quelle für strahlende Haut & Beauty Ti...,12500,52,122,...,2025-03-03T20:04:28.000Z,Nach 30 min nach der Reinigung spannt es total...,1,3.0,sophiak_2002,German,"verrate dir heute, wie du deinen Hauttyp herau...","In this video, a professional cosmetologist ex...",2025-02-17,de
30457,7472277773537529110,Deinen Hauttypen solltest du unbedingt kennen ...,de,2025-02-17 06:54:17+00:00,False,MIA 🦋 I Skincare & Glow ups,🫧 Deine Quelle für strahlende Haut & Beauty Ti...,12500,52,122,...,2025-02-21T13:08:33.000Z,Hast du auch einen Ratschlag für Lippen die im...,1,1.0,yvonne.stein5,German,"verrate dir heute, wie du deinen Hauttyp herau...","In this video, a professional cosmetologist ex...",2025-02-17,de
30458,7472277773537529110,Deinen Hauttypen solltest du unbedingt kennen ...,de,2025-02-17 06:54:17+00:00,False,MIA 🦋 I Skincare & Glow ups,🫧 Deine Quelle für strahlende Haut & Beauty Ti...,12500,52,122,...,2025-02-17T22:11:33.000Z,trockene haut,1,2.0,posarvhyv42,German,"verrate dir heute, wie du deinen Hauttyp herau...","In this video, a professional cosmetologist ex...",2025-02-17,de
30459,7472277773537529110,Deinen Hauttypen solltest du unbedingt kennen ...,de,2025-02-17 06:54:17+00:00,False,MIA 🦋 I Skincare & Glow ups,🫧 Deine Quelle für strahlende Haut & Beauty Ti...,12500,52,122,...,2025-02-21T10:15:00.000Z,ähmm ....Ich bin mir nicht so sicher\nIch habe...,0,3.0,yvonne.stein5,German,"verrate dir heute, wie du deinen Hauttyp herau...","In this video, a professional cosmetologist ex...",2025-02-17,de
